In [35]:
#!pip install pandas nltk
#!pip install tqdm

In [1]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
input_file = "../data/toxic_comments_cleaned.csv"
output_file = "../data/processed.csv"


Dataset should be downloaded from Here(https://www.kaggle.com/datasets/apoorvyadav/toxiccommentscleaned)

In [2]:
df = pd.read_csv('../data/toxic_comments_cleaned.csv')

C:\Users\msi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,id,comment_text,target
0,0000997932d777bf,explanation why the edits made under my userna...,0.0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0.0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0.0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impr...,0.0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11912547 entries, 0 to 11912546
Data columns (total 3 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id            object 
 1   comment_text  object 
 2   target        float64
dtypes: float64(1), object(2)
memory usage: 272.7+ MB


In [5]:
(df[df["target"] == 1]["id"].count()) / df["id"].count()

0.0140022112819366

In [6]:

# Download required resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\msi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
def clean_text(text):
    try:
        if isinstance(text, str):  # Ensure text is a string
            # Remove special characters, numbers, and extra spaces
            text = re.sub(r'[^a-zA-Z\s]', '', text)

            # Convert text to lowercase
            text = text.lower()

            # Tokenize text
            words = word_tokenize(text)

            # Remove stopwords
            stop_words = set(stopwords.words('english'))
            words = [word for word in words if word not in stop_words]

            # Apply stemming
            stemmer = PorterStemmer()
            stemmed_words = [stemmer.stem(word) for word in words]

            return ' '.join(stemmed_words)

        return ''  # Return empty string if not a string

    except Exception as e:
        return ''  # Return empty string in case of any error


In [8]:
clean_text(df['comment_text'].iloc[3])


'cant make real suggest improv wonder section statist later subsect type accid think refer may need tidi exact format date format later noon els first prefer format style refer want pleas let know appear backlog articl review guess may delay review turn list relev form exampl wikipediagoodarticlenominationstransport'

|

In [51]:
# Load dataset in chunks
chunk_size = 100  # Process 1000 rows at a time
input_file = "../data/toxic_comments_cleaned.csv"
output_file = "../data/processed.csv"

# Read CSV in chunks
chunks = pd.read_csv(input_file, chunksize=chunk_size)

# Enable tqdm progress bar
tqdm.pandas()

# Open output file to store processed data
with open(output_file, 'w', encoding='utf-8') as f_out:
    first_chunk = True  # To write headers only once

    for chunk in tqdm(chunks, desc="Processing chunks"):
        # Apply text cleaning function (NO recursion issues)
        chunk['cleaned_comment'] = chunk['comment_text'].apply(clean_text)

        # Append to CSV (write header only for the first chunk)
        chunk.to_csv(f_out, index=False, header=first_chunk, mode='a', encoding='utf-8',line_terminator='')
        first_chunk = False  # Ensure only the first chunk writes headers

Processing chunks: 20319it [41:26,  8.17it/s]


In [49]:
clean_text(df['comment_text'].iloc[115606])


''

In [55]:
df['cleaned_comment'].isna().sum()


8410

empty_count = df['cleaned_comment'].isna().sum()


In [15]:
df = df[(df['comment_text'].apply(lambda x: len(str(x).split())) <= 200) &
        (df['comment_text'].apply(lambda x: len(str(x)) <= 2000))]


Removing empty lines and lines with empty cleaned comments

In [56]:
df = pd.read_csv(output_file)

# Drop empty rows
df = df.dropna(how='all')
df = df.dropna(subset=['cleaned_comment'])

# Save the cleaned CSV
df.to_csv(output_file, index=False)

C:\Users\msi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Tokenization cleaned comments

In [57]:
nltk.download('punkt', quiet=True)

# Tokenize all comments
df['tokenized_comment'] = df['cleaned_comment'].apply(word_tokenize)

# Show example
print(df['tokenized_comment'].head())


0    [explan, edit, made, usernam, hardcor, metalli...
1    [daww, match, background, colour, im, seemingl...
2    [hey, man, im, realli, tri, edit, war, guy, co...
3    [cant, make, real, suggest, improv, wonder, se...
4               [sir, hero, chanc, rememb, page, that]
Name: tokenized_comment, dtype: object


In [16]:
df.to_csv(output_file, index=False)


In [79]:
# Build vocabulary from tokenized text
word_counts = Counter(word for tokens in df['tokenized_comment'] for word in tokens)

# Create word-to-index mapping (NO words are removed)
word2id = {word: idx+1 for idx, word in enumerate(word_counts.keys())}  # +1 to reserve 0 for padding


In [81]:
word2id

{'explan': 1,
 'edit': 2,
 'made': 3,
 'usernam': 4,
 'hardcor': 5,
 'metallica': 6,
 'fan': 7,
 'revert': 8,
 'werent': 9,
 'vandal': 10,
 'closur': 11,
 'ga': 12,
 'vote': 13,
 'new': 14,
 'york': 15,
 'doll': 16,
 'fac': 17,
 'pleas': 18,
 'dont': 19,
 'remov': 20,
 'templat': 21,
 'talk': 22,
 'page': 23,
 'sinc': 24,
 'im': 25,
 'retir': 26,
 'daww': 27,
 'match': 28,
 'background': 29,
 'colour': 30,
 'seemingli': 31,
 'stuck': 32,
 'thank': 33,
 'januari': 34,
 'coordin': 35,
 'univers': 36,
 'time': 37,
 'hey': 38,
 'man': 39,
 'realli': 40,
 'tri': 41,
 'war': 42,
 'guy': 43,
 'constantli': 44,
 'relev': 45,
 'inform': 46,
 'instead': 47,
 'seem': 48,
 'care': 49,
 'format': 50,
 'actual': 51,
 'info': 52,
 'cant': 53,
 'make': 54,
 'real': 55,
 'suggest': 56,
 'improv': 57,
 'wonder': 58,
 'section': 59,
 'statist': 60,
 'later': 61,
 'subsect': 62,
 'type': 63,
 'accid': 64,
 'think': 65,
 'refer': 66,
 'may': 67,
 'need': 68,
 'tidi': 69,
 'exact': 70,
 'date': 71,
 'noon':

In [83]:
print("Total vocabulary size:", len(word2id))
print("Example word mappings:", list(word2id.items())[:10])

Total vocabulary size: 612571
Example word mappings: [('explan', 1), ('edit', 2), ('made', 3), ('usernam', 4), ('hardcor', 5), ('metallica', 6), ('fan', 7), ('revert', 8), ('werent', 9), ('vandal', 10)]


In [84]:
# Convert tokenized text to numerical sequences
df['sequence'] = df['tokenized_comment'].apply(lambda tokens: [word2id[word] for word in tokens])

# Show example
print(df[['tokenized_comment', 'sequence']].head())


                                   tokenized_comment  \
0  [explan, edit, made, usernam, hardcor, metalli...   
1  [daww, match, background, colour, im, seemingl...   
2  [hey, man, im, realli, tri, edit, war, guy, co...   
3  [cant, make, real, suggest, improv, wonder, se...   
4             [sir, hero, chanc, rememb, page, that]   

                                            sequence  
0  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...  
1  [27, 28, 29, 30, 25, 31, 32, 33, 22, 34, 35, 3...  
2  [38, 39, 25, 40, 41, 2, 42, 43, 44, 20, 45, 46...  
3  [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 6...  
4                           [91, 92, 93, 94, 23, 95]  


In [85]:
df.to_csv(output_file, index=False)


In [9]:
df = pd.read_csv(output_file)

C:\Users\msi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
max_word_count = max(df['comment_text'].apply(lambda x: len(str(x).split())))
print(max_word_count)


3750


In [19]:
max_index = df['comment_text'].apply(lambda x: len(str(x).split())).idxmax()
print(df.loc[max_index, 'comment_text'])  # Prints the comment with max words


 all of those statements need to be referenced  leaving an article unreferenced in every section opens the door for original research and the introduction for factually inaccurate information  to point out one thing ill bring up consolidated appropriations act 2012  first it has a number of tags on it the creation of articles that need to be tagged should be reason number 1 to remove the flag  second the only referencelink provided on the article is this one while the information is from various subpages of that  it appears that the entirety of the article is found at this deeper link  its possible that some came from this as well i cant tell  ill also note that the style of the summary it was taken from was never meant to be encyclopedic so the article is riddled with grammatical errors mostly incomplete sentences  on creation the opening read consolidated appropriations act 2012 hour 2055 is a bill passed to the 112th united states congress  ive never heard this usage to rather than 

In [ ]:
import numpy as np

def pad_sequence(seq, max_length):
    return seq + [0] * (max_length - len(seq))  # Pad with 0

df['padded_sequence'] = df['sequence'].apply(lambda seq: pad_sequence(seq, max_length))

# Convert to NumPy array for model training
X = np.array(df['padded_sequence'].tolist())

print("Final shape of input data:", X.shape)  # Should be (num_samples, max_length)


In [17]:
# Find the index of the longest comment
longest_comment_idx = df['sequence'].apply(len).idxmax()

# Get the longest comment and its length
longest_comment = df.loc[longest_comment_idx, 'comment_text']
longest_comment_length = len(df.loc[longest_comment_idx, 'sequence'])

print("Longest Comment:", longest_comment)
print("Length of Longest Comment (in tokens):", df.loc[longest_comment_idx, 'sequence'])


Longest Comment: poooooooooooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooo

In [18]:
# Calculate the lengths of the comments and store them as a list of tuples (length, comment)
comment_lengths = [(len(seq), comment) for seq, comment in zip(df['sequence'], df['comment_text'])]

# Sort by length in descending order and get the top 10 longest comments
top_10_longest_comments = sorted(comment_lengths, key=lambda x: x[0], reverse=True)[:10]

# Print the top 10 longest comments and their lengths
for length, comment in top_10_longest_comments:
    print(f"Length: {length}, Comment: {comment[:200]}...")  # Print the first 200 characters of the comment for readability


Length: 1472, Comment: poooooooooooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ooooooooo ...
Length: 1424, Comment: mis horny horny  horny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny hornyhorny h...
Length: 1303, Comment: problem    beschwerdefhrer benutzerwhiggsgerm    beteiligte administratoren benutzerinitti und benutzerjkb    missbruchlich eingesetzte funktion benutzersperre    erluterung  ohne bzw mit falscher und...
Length: 1267, Comment:  lied der deutschen  nazilied     lieber prohibitonions    schade da du in der englischen ausgabe der wikipedia unverantwortliches unwissen verbreitest die nationalhymne war wie durch den schriftverke...
Length: 1254, Comment:  o lucilio  um filho da puta     lucilio filho da puta lucilio filho 